In [1]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from pylab import rcParams

In [14]:
from keras.preprocessing.sequence import pad_sequences
#function that convert a char seqs to numbers seqs 
#(it does a little more but lets leave it for now)
def encode_sequence_list(seqs, maxlen, feat_n=0):
    encoded_seqs = []
    for seq in seqs:
        encoded_seq = [char_to_int[c] for c in seq]
        encoded_seqs.append(encoded_seq)
    if(feat_n > 0):
        encoded_seqs.append(np.zeros(feat_n))
    return pad_sequences(encoded_seqs, padding='post', maxlen=maxlen)

def decode_sequence_list(seqs):
    decoded_seqs = []
    for seq in seqs:
        decoded_seq = [int_to_char[i] for i in seq]
        decoded_seqs.append(decoded_seq)
    return decoded_seqs

In [44]:
filename='sm_15_lettersdim_75acc.h5'
dim = 15

In [79]:
from keras.models import load_model

autoencoder = load_model(filename)

In [45]:
letters = ['ṭ', 'j', 'V', 'ū', 'p', 'y', 'l', 'ť', 'z', 'ṇ',
           'v', 't', 's', 'h', 'e', 'ś', 'ą', 'ṣ', 'ṃ', 'ṝ',
           'a', 'Z', 'o', 'g', 'ṛ', 'm', 'ñ', 'ḍ', 'ā', 'ḥ', 'S', 'b', 'i', 'u',
           'k', 'c', 'ī', 'd', 'r', 'n', '˛', 'ṅ']
char_to_int = dict((c, i) for i, c in enumerate(letters))
int_to_char = dict((i, c) for i, c in enumerate(letters))
len(letters)

42

In [78]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = MinMaxScaler()
wlist = ['sevyatām','ditisutāḥ', 'thomas']
seqs_ds = pd.DataFrame(wlist)
encoded_seqs = encode_sequence_list(wlist, dim)
scaled_seqs = scaler.fit_transform(encoded_seqs)

In [80]:
#get the MSE error term
predictions = autoencoder.predict(scaled_seqs)
#mse = np.mean(np.power(scaled_seqs - predictions, 2), axis=1)
#print('MSE:', np.quantile(mse, 0.999)) # => the 9999% quatile - only 0.0001% have error score higher than that

In [84]:
#encode all the data
encoded_seqs = encode_sequence_list(seqs_ds.iloc[:,0],dim)
#scale it
scaled_data = MinMaxScaler().fit_transform(encoded_seqs)
#predict it
predicted = autoencoder.predict(scaled_data)
#get the error term
mse = np.mean(np.power(scaled_data - predicted, 2), axis=1)
#now add them to our data frame
seqs_ds['MSE'] = mse

In [85]:
display(seqs_ds)

,0,MSE,MSE_Outlier
0,sevyatām,0.006087,1
1,ditisutāḥ,0.004921,0
2,thomas,0.001726,0


In [83]:
mse_threshold = np.quantile(seqs_ds['MSE'], 0.9999)
print(f'MSE 0.9999 threshhold:{mse_threshold}')
seqs_ds['MSE_Outlier'] = 0
seqs_ds.loc[seqs_ds['MSE'] > mse_threshold, 'MSE_Outlier'] = 1
print(f"Num of MSE outlier:{seqs_ds['MSE_Outlier'].sum()}")

seqs_ds.loc[seqs_ds['MSE_Outlier'] == 1]

MSE 0.9999 threshhold:0.006086682941338582
Num of MSE outlier:1


,0,MSE,MSE_Outlier
0,sevyatām,0.006087,1
